# Iskhakov and Keane (2018)

In [1]:
import numpy as np
import numba as nb
import pandas as pd

In [2]:
HOURS = np.array([0, 20, 40, 45, 50, 60])
RELATIVE_HOURS = HOURS / HOURS.max()

In [11]:
@nb.njit
def compute_period_human_capital(hc_previous_period, choices, period):
    container = []
    for hc in hc_previous_period:
        for choice in choices:
            updated_hc = 1 / (period  + 1) * (hc * period + choice)
            if updated_hc not in container:
                container.append(updated_hc)
                
    return np.array(container)

def compute_unique_human_capitals(choices, n_periods):
    container = [np.zeros(1)]

    for period in range(n_periods - 1):
        human_capitals = compute_period_human_capital(container[period], choices, period)
        
        container.append(human_capitals)
        
    return container

def create_state_space(n_periods, choices):
    per_period_human_capitals = compute_unique_human_capitals(choices, n_periods)
    
    to_concat = []
    for period, human_capitals in enumerate(per_period_human_capitals):
        df = pd.DataFrame({"period": period, "human_capital": human_capitals})
        to_concat.append(df)
        
    df = pd.concat(to_concat)
    
    return df

In [12]:
create_state_space(40, RELATIVE_HOURS)

,period,human_capital
0,0,0.000000
0,1,0.000000
1,1,0.333333
2,1,0.666667
3,1,0.750000
...,...,...
8705,39,0.991453
8706,39,0.989316
8707,39,0.993590
8708,39,0.995726
